# CORDA tutorial

## Reactions, Genes and confidences

In order to start a reconstruction CORDA requires you to assign a confidence score to each reaction in your base model. This can be done by a variety of methods, and even by hand, but the most common way is to assign confidence based on proteome or gene expression data.

CORDA manages a total of 5 confidence levels:

- -1 for reactions that are not present and should not be included in the model
- 0 for reactions with unknown confidence which may be included in the model if necessary
- 1 for low confidence reactions that should be included if necessary
- 2 for medium confidence reactions that should be included if necessary
- 3 for high confidence reactions that must be included if possible in any way

The most tedious step here is usaully mapping the confidence for genes or proteins to the distinct reactions. Many of the larger models come with gene-reaction rules in the form

*gene1 and gene2 or (gene3 and gene4)* 

and the individual confidence values have to be mapped from the gene confidence levels. Here "and" is evaluated by the minimum confidence and "or" by the maximum confidence. The Python package includes a handy function to do this for you automatically in a safe manner. For that you will require the gene-reaction rule (Recon 1 and 2 include them in their model for instance) and a dictionary mapping genes/proteins to their confidence values. For examples:

In [1]:
from corda import reaction_confidence

gene_conf = {"gene1": 1, "gene2": 3, "gene4": -1} # missing entries are automatically assigned zeroes
rule = "gene1 and gene2 or (gene3 and gene4)"

reaction_confidence(rule, gene_conf)

1

## A small example

Now we will perform a small reconstruction. The package includes a small model in SBML format describing the general central carbon metabolism. 

In [2]:
from corda import test_model

mod = test_model()
len(mod.reactions)

60

Here the last reaction is a biomass reaction.

In [3]:
mod.reactions[59].reaction

'0.39253 3pg + 20.7045 atp + 0.15446 cit + 0.38587 glu + 0.35261 oaa + 0.053446 prpp + 0.50563 pyr --> 20.6508 adp + 20.6508 pi'

We can now reconstruct the smallest model that still allows for the production of biomass. Let's start by setting the biomass reaction as high confidence and all other reactions as "not include".

In [4]:
conf = {}
for r in mod.reactions: conf[r.id] = -1
conf["r60"] = 3

Now we reconstruct the model.

In [5]:
from corda import CORDA

opt = CORDA(mod, conf)
opt.build()
print(opt)

build status: reconstruction complete
Inc. reactions: 33/101
 - unclear: 0/0
 - exclude: 32/100
 - low and medium: 0/0
 - high: 1/1



The metric you see are for irreversible reactions. We can obtain the reversible reconstruction using:

In [6]:
rec = opt.cobra_model(name="minimum")
len(rec.reactions)

33

This gives a reconstruction that looks like this (red denotes included reactions):
![reconstruction](reconstruction.png)

We can also define additional metabolic functions that should be possible in the reconstruction. Let's assume we want to be able to produce succinate.

In [7]:
opt = CORDA(mod, conf, met_prod="succ")
opt.build()
print(opt)

build status: reconstruction complete
Inc. reactions: 35/102
 - unclear: 0/0
 - exclude: 33/100
 - low and medium: 0/0
 - high: 2/2



This time we included the production of succinate via fumarate:

In [10]:
rec = opt.cobra_model("plus_succ")
print(len(rec.reactions))
use = rec.metabolites.fum.reactions
for r in use: print(r.reaction)

34
fum <=> mal_L
succ <=> fum
